<a href="https://colab.research.google.com/github/haruhiysd/QM-Group-Project/blob/main/QM_group_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install Basemap
!pip install ipyleaflet

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pylab
from datetime import datetime

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10, 8)

In [9]:
!pip install folium ipywidgets pandas
import folium
import json

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display


In [5]:
df_wide = pd.read_csv("/Number of tourists.csv", thousands=",")
df_wide

,Year,Spain total,Andalucía,Aragón,"Asturias, Principado de","Balears, Illes",Canarias,Cantabria,Castilla y León,Castilla - La Mancha,Cataluña,Comunitat Valenciana,Extremadura,Galicia,"Madrid, Comunidad de","Murcia, Región de","Navarra, Comunidad Foral de",País Vasco,"Rioja, La"
0,2016,75315008,10589642,644516,277989,12997549,13259567,378435,1225331,222651,18139177,7731770,261453,1035147,5783137,924305,300622,1382083,150779
1,2017,81868522,11518262,562352,294129,13792296,14214222,414489,1458546,226221,19118421,8925959,380914,1291086,6699785,991209,333317,1514765,124189
2,2018,82808413,11681256,545530,299357,13851598,13752022,381181,1374464,235011,19196344,9206908,468286,1512511,7139775,1134189,323730,1552389,142926
3,2019,83509153,12023153,544761,317860,13679781,13146863,392516,1286198,262044,19375152,9535496,469398,1624711,7640980,1037818,415054,1611280,139497
4,2020,18933103,2700613,172084,92222,1721123,3787228,106392,430405,95887,3870259,2542758,156169,540004,1718335,279245,146325,527286,45355
5,2021,31180802,4248340,215884,161462,6324711,5258729,159960,499370,98134,5787837,4019766,218790,772452,2177068,385228,123614,668683,47913
6,2022,71659281,10009890,385175,277267,13203885,12328012,323694,1016508,200063,14852042,8616445,319504,1271894,6048182,821232,273406,1545013,105910
7,2023,85169050,12190565,426796,375560,14424512,13950687,434110,1217204,217062,18182612,10474167,280110,1616929,7846067,976513,363686,2002173,118208


In [7]:

region_cols = [col for col in df_wide.columns if col not in ["Year", "Spain total"]]

df_long = df_wide.melt(
    id_vars="Year",
    value_vars=region_cols,
    var_name="region",
    value_name="tourists"
)

df_long


,Year,region,tourists
0,2016,Andalucía,10589642
1,2017,Andalucía,11518262
2,2018,Andalucía,11681256
3,2019,Andalucía,12023153
4,2020,Andalucía,2700613
...,...,...,...
131,2019,"Rioja, La",139497
132,2020,"Rioja, La",45355
133,2021,"Rioja, La",47913
134,2022,"Rioja, La",105910


In [18]:
with open("/georef-spain-comunidad-autonoma.geojson", "r", encoding="utf-8") as f:
    spain_geojson = json.load(f)


In [21]:
def create_map(year, selected_regions):

    df_year = df_long[df_long["Year"] == year].copy()

    if selected_regions:
        df_year = df_year[df_year["region"].isin(selected_regions)]

    m = folium.Map(location=[40.0, -3.0], zoom_start=6)

    if not df_year.empty:
        folium.Choropleth(
            geo_data=spain_geojson,
            data=df_year,
            columns=["region", "tourists"],
            key_on="features",
            fill_color="YlOrRd",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f"Number of Tourists ({year})"
        ).add_to(m)
    else:
        folium.Marker(location=[40.0, -3.0], popup="No Data").add_to(m)

    folium.GeoJson(
        spain_geojson,
        name="Regions",
        tooltip=folium.GeoJsonTooltip(
            fields=["NAME_1"],
            aliases=["Region:"],
            localize=True
        ),
    ).add_to(m)

    return m


In [22]:
unique_years = sorted(df_long["Year"].unique())
unique_regions = sorted(df_long["region"].unique())
year_widget = widgets.Dropdown(
    options=unique_years,
    value=unique_years[-1],
    description='Year:'
)

regions_widget = widgets.SelectMultiple(
    options=unique_regions,
    value=(),
    description='Regions (multi):',
    rows=10
)
interactive_map = interactive(
    create_map,
    year=year_widget,
    selected_regions=regions_widget
)

display(interactive_map)


interactive(children=(Dropdown(description='Year:', index=7, options=(2016, 2017, 2018, 2019, 2020, 2021, 2022…